# DEL 3.6 - URL-encoding


In [1]:
# Importer bibliotekene vi trenger
import requests
import json
from urllib.parse import quote, quote_plus, urlencode, unquote, unquote_plus, parse_qs



Før vi går løs på neste oppgave, skal vi raskt se litt på url-encoding. Dette er
en metode for å konvertere spesialtegn til et format som kan sendes trygt over
internett. Siden URLer kun kan inneholde et begrenset sett med ASCII-tegn, må
andre tegn – som æ, ø, å, mellomrom, og spesialsymboler – kodes om.

Tegn som ikke er ASCII erstattes med % etterfulgt av to heksadesimale siffer som
representerer tegnets byte-verdi. Noen andre tegn som har spesiell betydning i
url-er, som `?` og `=` kodes også om, i tillegg til mellomrom som ikke
aksepteres i url-er. Eksempel på konverteringer:

- Mellomrom → `%20` (eller `+` i noen kontekster)
  - `+` brukes ofte for søkestrenger, og gir en penere url. Eks: [https://www.google.com/search?q=sandnes+sentrum](https://www.google.com/search?q=sandnes+sentrum)
- `å` → `%C3%A5`
- `?` → `%3F`

> æøå i url-er
> 
> Moderne nettlesere og systemer kan håndtere spesielle tegn som ææå i url-er.
> Men det kan gi kompabilitesutfordringer, og er i praksis lite vanlig. Eneste
> jeg fant eksempel på nå er [rødt.no](https://rødt.no). Da er det uansett
> vanlig å ha et annet domene i tillegg, kun med ascii-tegn (som rødt.no og
> roedt.no)
>
> Spesialtegn må/bør uansett unnlates i kode. Det er nettleseren som konverterer
> spesialtegn før DNS-request sendes, så selv når du går til rødt.no, konverteres
> dette til [xn--rdt-0na.no](https://xn--rdt-0na.no) før det sendes videre.

**URL-encoding i python**
Python har et standard-bibliotek som heter `urllib`, og i modulen `urllib.parse` 
finnes det funksjoner for URL parsing, encoding og decoding. Dette inkluderer
behandling av _query parameters_ i url-er. 

> Litt usikker på hva _query parameters_ er?
>
> URL-er består av mange deler. Til slutt kan den ha argumenter som sendes til
> server, som kommer etter `?` og kan se slik ut: `/search?q=url%20explained`
> Her sendes parameteret `q` med verdien `'url explained'`
> Les mer om url-er på
> [mdn](https://developer.mozilla.org/en-US/docs/Learn_web_development/Howto/Web_mechanics/What_is_a_URL)

De viktigste funksjonene fra `urllib.parse` er

_For tekst til url-format_
- `quote` - koder en tekstverdi til url sikkert format
- `quote_plus` - samme som quote, men mellomrom kodes som `+` i stedet for `%20`
- `urlencode` - Koder `dict` som query-string

_For url-format til tekst_
- `unquote` - dekoder url-format til tekstverdi
- `unquote_plus` - samme som unquote, men forventer mellomrom kodet som `+`
- `parse_qs` - Henter ut data fra query-string til `dict`

**Eksempler på url-koding i python**


In [2]:


# quote() - Koder en enkelt streng
tekst = "Hei på deg!"
encoded = quote(tekst)
print("quote()")
print("-------------")
print(f"{tekst} -> {encoded}")

# quote_plus() - Bruker + for mellomrom (vanlig i forms)
encoded_plus = quote_plus(tekst)
print("\nquote_plus()")
print("-------------")
print(f"{tekst} -> {encoded_plus}")

# urlencode() - Koder dictionary til query string
params = {
    'søk': 'python programmering',
    'side': 1,
    'filter': 'nye populære'
}
query_string = urlencode(params)
print("\nurlencode()")
print("-------------")
print("params dictionary:")
print(json.dumps(params, indent=2, ensure_ascii=False))
print(f"query_string: {query_string}")

quote()
-------------
Hei på deg! -> Hei%20p%C3%A5%20deg%21

quote_plus()
-------------
Hei på deg! -> Hei+p%C3%A5+deg%21

urlencode()
-------------
params dictionary:
{
  "søk": "python programmering",
  "side": 1,
  "filter": "nye populære"
}
query_string: s%C3%B8k=python+programmering&side=1&filter=nye+popul%C3%A6re


**Eksempler på url-dekoding i python**

In [3]:

# Dekode URL-encoded strenger
encoded = "Hei%20p%C3%A5%20deg%21"
decoded = unquote(encoded)
print("\nunquote()")
print("-------------")
print(f"{encoded} -> {decoded}")

# Dekode med + som mellomrom
encoded_plus = "Hei+p%C3%A5+deg%21"
decoded_plus = unquote_plus(encoded_plus)
print("\nunquote_plus()")
print("-------------")
print(f"{encoded_plus} -> {decoded_plus}")

# Parse query string tilbake til dictionary
query_string = "s%C3%B8k=python+programmering&side=1&filter=nye+popul%C3%A6re"
parsed_params = parse_qs(query_string)
print("\nparse_qs()")
print("-------------")
print(f"{query_string} ->")
print(json.dumps(parsed_params, indent=2, ensure_ascii=False))


unquote()
-------------
Hei%20p%C3%A5%20deg%21 -> Hei på deg!

unquote_plus()
-------------
Hei+p%C3%A5+deg%21 -> Hei på deg!

parse_qs()
-------------
s%C3%B8k=python+programmering&side=1&filter=nye+popul%C3%A6re ->
{
  "søk": [
    "python programmering"
  ],
  "side": [
    "1"
  ],
  "filter": [
    "nye populære"
  ]
}


**Eksempel på bygging av komplett url i python**

In [4]:
base_url = "https://api.example.com/search"
params = {
    'q': 'machine learning',
    'lang': 'no',
    'år': 2024
}
full_url = f"{base_url}?{urlencode(params)}"
print("\nFull URL med query parameters:")
print(full_url)


Full URL med query parameters:
https://api.example.com/search?q=machine+learning&lang=no&%C3%A5r=2024


**Query params med request**

Det er ikke alltid nødvendig å gjøre konvertering med `urlencode` før
man for eksempel skal hente data med `request`. Funksjoner i `request`
kan gjøre denne konverteringen for deg automatisk:

In [5]:
# Eksempel på bruk med requests, med et test API som støtter query parameters
# Vi skal hente objekter med id 1, 2 og 3 ved å sende dem som query parameters
base_url = "https://api.restful-api.dev/objects"
ids = [1, 2, 3]

resultater = []

# Metode 1: Bruk params dict med liste som verdi, brukes direkte i requests.get
params = {
    'id': ids,
}
response = requests.get(base_url, params=params)

resultater.append({
    'qs': response.request.url.split('?')[1],
    'data': response.json()
})


# Metode 2: Bruk urlencode med doseq=True for å håndtere liste som verdi
qs_urlencode_doseq = urlencode(params, doseq=True)
url_with_qs_doseq = f"{base_url}?{qs_urlencode_doseq}"

response_doseq = requests.get(url_with_qs_doseq)

resultater.append({
    'qs': qs_urlencode_doseq,
    'data': response_doseq.json()
})

# Metode 3: Manuell konstruksjon av query string med kommaseparert liste
params_csv = {
    'id': ','.join(map(str, ids)),  # Konverterer liste til kommaseparert streng
}
query_string_csv = urlencode(params_csv)
url_with_qs_csv = f"{base_url}?{query_string_csv}"

response_csv = requests.get(url_with_qs_csv)

resultater.append({
    'qs': query_string_csv,
    'data': response_csv.json()
})


# Utskrift av resultater
print("Query strings fra de ulike metodene:")
print('-' * 35)
print(f"| {'Metode':<10}| {'Query String':<20}|")
print(f"+{'-'*11}+{'-'*21}+")
for i, resultat in enumerate(resultater, start=1):
    print(f"| {'Metode ' + str(i):<10}| {resultat['qs']:<20}|")
print('-' * 35)

# Sjekk at alle metodene gir samme data
resultat_1 = json.dumps(resultater[0]['data'], sort_keys=True)
data_like = True
for resultat in resultater[1:]:
    resultat_n = json.dumps(resultat['data'], sort_keys=True)
    if resultat_n != resultat_1:
        data_like = False
        break

if data_like:
    print("\nData fra alle metodene samsvarer!")
    print("Responsdata:")
    print(json.dumps(resultater[0]['data'], indent=2))
else:
    print("\nData fra metodene samsvarer ikke...")
    for i, resultat in enumerate(resultater, start=1):
        print(f"\nData fra metode {i}:")
        print(json.dumps(resultat['data'], indent=2))

Query strings fra de ulike metodene:
-----------------------------------
| Metode    | Query String        |
+-----------+---------------------+
| Metode 1  | id=1&id=2&id=3      |
| Metode 2  | id=1&id=2&id=3      |
| Metode 3  | id=1%2C2%2C3        |
-----------------------------------

Data fra alle metodene samsvarer!
Responsdata:
[
  {
    "id": "1",
    "name": "Google Pixel 6 Pro",
    "data": {
      "color": "Cloudy White",
      "capacity": "128 GB"
    }
  },
  {
    "id": "2",
    "name": "Apple iPhone 12 Mini, 256GB, Blue",
    "data": null
  },
  {
    "id": "3",
    "name": "Apple iPhone 12 Pro Max",
    "data": {
      "color": "Cloudy White",
      "capacity GB": 512
    }
  }
]
